# Gradio App answering questions about a pdf file or a file uploaded from the arXiv website

Necessary libraries and script to download the model

In [ ]:
import os
from huggingface_hub import login
from llama_cpp import Llama
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter 
from langchain_community.document_loaders import PyPDFLoader, ArxivLoader
import chromadb
import gradio as gr

token = os.getenv('token_hf')  #HuggingFace access token
login(token=token)

llm = Llama.from_pretrained(
	repo_id="microsoft/Phi-3-mini-4k-instruct-gguf",
	filename="Phi-3-mini-4k-instruct-q4.gguf",
    n_ctx = 4096,  # Context window length
    n_threads = 8, # The number of CPU threads to use, tailor to your system and the resulting performance
    n_gpu_layers=0, # The number of layers to offload to GPU, set to 0 if no GPU acceleration is available
)

## Prompt Template for the model

In [19]:
#Function which returns the answer of the model to a query
def llm_output(query: str)->str:
    output = llm(
      f"<|user|>\n{query}<|end|>\n<|assistant|>", #format for the prompt
      max_tokens=350,  #maximum number of tokens generated by the model
      stop=["<|end|>"], 
      echo=False,  # Whether to echo the prompt
      temperature=0.3
    )
    return output['choices'][0]['text'] #output is a dictionary. We just retain the text of the provided answer

## Auxiliary functions to handle RAG for a pdf file

The pdf file is split into chunks which are then stored in the vector database

In [20]:
# We load a pdf file with PyPDFLoader. The function returns a list of documents (one per page)
def document_loader(file)->list[dict[str]]:
    loader=PyPDFLoader(file)
    loaded_document=loader.load_and_split()
    return loaded_document 

#We split the loaded pdf into chunks of text. Returns a list of documents (one for each chunk)
def text_splitter(data: list[dict[str]])->list[dict[str]]:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=40,
        separators=["\n\n", ""], #this instead of the default separators to improve chunk overlaps
        length_function=len,
    )
    chunks = text_splitter.split_documents(data)
    return chunks

#Returns the 3 chunks of text most relevant for the query, as determined by chromadb
def doc_database(chunks: list[dict[str]], query: str)->list[str]:
    
    #chunk_list retains for each chunk only the text (page_content) and removes metadata. To be used for the Chroma collection
    chunk_list=list(map(lambda x: x.page_content, chunks))  
    
    #List of ids for the Chroma collection (list of strings)
    ids=list(map(str,range(len(chunks))))
    
    client=chromadb.Client()
    collection=client.get_or_create_collection(
        name='doc_collection',
        configuration={
            "hnsw": {
                "space": "cosine" #The default metric is L^2 but cosine is more suited for text similarity
            }
        }
    )
    collection.upsert(ids=ids, documents=chunk_list)
    results= collection.query(query_texts=[query], n_results=5)  
    return results['documents'][0] #results['documents'] is a list with a sinle element, which is the list of relevant chunks

## Auxiliary functions to handle RAG for a document uploaded from the arXiv website

The document is uploaded from the arXiv website and split into chunks stored in the vector Database

In [21]:
# Similar to the document_loader function defined previously. We load a file from the arXiv website by providing the arxiv number (xxxx.xxxxx) as input.
def arxiv_loader(arxiv_num: str)->list[dict[str]]:
    loader=ArxivLoader(arxiv_num)
    loaded_document=loader.load()
    return loaded_document 

#Variant of the text_splitter function defined previously to handle arxiv documents
def arxiv_text_splitter(data: list[dict[str]])-> list[str]:
    text=data[0].page_content  #The output of arxiv_loader is a list with one element. We extract the content which is a string
    text_splitter = CharacterTextSplitter( #since text is of type str, without separators, we can use CharacterTextSplitter
        chunk_size=500,
        chunk_overlap=40,
        separator='', 
        length_function=len,
    )
    chunks = text_splitter.split_text(text)
    return chunks

#Variant of the doc_database function tailored for the arxiv documents. Returns the 5 most relevant chunks of text.
def arxiv_database(chunks: list[str], query: str)->list[str]:
    
    #We do not need to preprocess chunks since in this case the output of arxiv_text_splitter is already a list of strings
    #List of ids for the Chroma collection (list of strings)
    ids=list(map(str,range(len(chunks))))
    
    client=chromadb.Client()
    collection=client.get_or_create_collection(
        name='arxiv_collection',
        configuration={
            "hnsw": {
                "space": "cosine"
            }
        }
    )
    collection.upsert(ids=ids, documents=chunks)
    results= collection.query(query_texts=[query], n_results=5)  
    return results['documents'][0] #results['documents'] is a list with a sinle element, which is the list of relevant chunks

## Answer generated by the model

The retrieved chunks of text are added to the template and provide the context which allow the model to answer

In [22]:
#Function which generates the model output. Works for both PDF and arXiv documents

def text_question(query: str, database: list[str])->str:
    '''
    inputs:
           query (the question we ask to the chatbot)
           database (the list of relevant text chunks from doc_database)
    outputs the answer of the chatbot
    '''
    #We append to the prompt template the text contained in database
    text=''
    for i in range(len(database)):
        text += database[i]
        
    template='Considering the following text, can you explain'+query+'text:'+text   
    output=llm_output(template)
    return output

## User Interface with gradio

In [ ]:
#Gradio application

#Function which combines all the previous ones to generate a chatbot answer about the pdf file from a query
def pdf_qa(query: str, file)->str:
    data = document_loader(file)
    chunks = text_splitter(data)
    database = doc_database(chunks,query)
    return text_question(query, database)

#Function which combines all the previous ones to generate a chatbot answer about the arxiv file from a query
def arxiv_qa(query: str, arxiv_num: str)->str:
    data = arxiv_loader(arxiv_num)
    chunks = arxiv_text_splitter(data)
    database = arxiv_database(chunks,query)
    return text_question(query, database)


#Code for gradio App. We include two buttons b1 and b2 which determine whether we have to process the uploaded pdf or the arXiv file respectively.
demo = gr.Blocks()

with demo:
    gr.Markdown(
    """
    # Document QA
    Upload a PDF File and press 'File upload' or provide the arXiv number and press 'File from arXiv'. The bot will answer your question.
    """)
    query  = gr.Textbox(label="Input Question", lines=2, placeholder="Type your question here...")
    file   = gr.File(label="Upload PDF File", file_count="single", file_types=['.pdf'], type="filepath")  # Drag and drop file upload
    text   = gr.Textbox(label="arXiv number", lines=2, placeholder="Enter a valid arXiv number")
    answer = gr.Textbox(label="Output")

    b1 = gr.Button("File upload")
    b2 = gr.Button("File from arXiv")

    b1.click(pdf_qa, inputs=[query, file], outputs=answer)
    b2.click(arxiv_qa, inputs=[query, text], outputs=answer)

demo.launch()